In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import CRPS.CRPS as pscore


pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.dumas_evaluation import *

In [2]:
log_name = 'test'
with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

/tmp/ipykernel_4161/2614114049.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
with open('./dumas_model.pickle', 'rb') as f:
    dumas_model = pickle.load(f)

In [4]:
import conduct_evaluation
#sample_dumas = SampleOutcomes_Dumas_Normal(test_event_log, dumas_model)

evaluator = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal, {},
                                     test_event_log, n=1000)

In [5]:
likelihoods = evaluator.sample_cases(plot_cases=False, multiprocessing=True)

100%|█| 1235/1235 [00:00<00:00, 1840.38it
100%|█| 1235/1235 [07:36<00:00,  2.71it/s
100%|██████████████████████| 1235/1235 [00:53<00:00, 23.09it/s]


In [6]:
np.mean([v.ln() for v in likelihoods[0].values()])

Decimal('0.1413094856579221573587854184')

In [7]:
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

np.mean(get_pscores(likelihoods))

np.float64(10752.85436477798)

In [8]:
results = {
    'dumas_model' : likelihoods,
}
with open('./pcr_dumas_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)